<a href="https://colab.research.google.com/github/keisyashakila/MSCI-151/blob/main/MILP_CW5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from collections import defaultdict
import os
import csv

Correcting import statements:
- `from collection import dafaultdict` to `from collections import defaultdict`
- Assuming `import css` was intended as `import csv` as `css` is not a standard Python module.

In [4]:
from collections import defaultdict
import os
import csv

In [1]:
!pip install pulp

In [2]:
import pulp as pl

In [6]:
baristas = ["Max", "Jiwa", "Fore", "Donna", "Paul"]
days = list(range(1,7))
blocks = list(range(1,5))
H = 4

days 1-7, due to 7 days a week

In [7]:
cost = {"Max": 50000, "Jiwa": 50000, "Fore": 50000, "Donna": 150000, "Paul": 150000}
etype = {"Max": "P", "Jiwa": "P", "Fore": "P", "Donna": "F", "Paul": "F"}
minWeekly_default = {"Max": 12, "Jiwa": 12, "Fore": 12, "Donna": 36, "Paul": 36}

In [8]:
avail = {("Max", 1): 8, ("Max", 2): 8, ("Max", 3): 8, ("Max", 4): 0, ("Max", 5): 8, ("Max", 6): 4, ("Max", 7): 4, ("Jiwa", 1): 4, ("Jiwa", 2): 4, ("Jiwa", 3): 8, ("Jiwa", 4): 8, ("Jiwa", 5): 0, ("Jiwa", 6): 4, ("Jiwa", 7): 0, ("Fore", 1): 8, ("Fore", 2): 0, ("Fore", 3): 8, ("Fore", 4): 8, ("Fore", 5): 8, ("Fore", 6): 8, ("Fore", 7): 0, ("Donna", 1): 12, ("Donna", 2): 12, ("Donna", 3): 12, ("Donna", 4): 12, ("Donna", 5): 12, ("Donna", 6): 12, ("Donna", 7): 8, ("Paul", 1): 12, ("Paul", 2): 8, ("Paul", 3): 12, ("Paul", 4): 12, ("Paul", 5): 8, ("Paul", 6): 12, ("Paul", 7): 12}

In [9]:
days = {1: "Monday", 2: "Tuesday", 3: "Wednesday", 4: "Thursday", 5: "Friday", 6: "Saturday", 7: "Sunday"}

In [10]:
blocks = {1: "07:00-11:00", 2: "11:00-15:00", 3: "14:00-15:00", 4: "18:00-23:00"}